#**DBT**
dbt is a transformation workflow that helps you get more work done while producing higher quality results. You can use dbt to modularize and centralize your analytics code, while also providing your data team with guardrails typically found in software engineering workflows. Collaborate on data models, version them, and test and document your queries before safely deploying them to production, with monitoring and visibility

###Snowflake setup:

creating the required warehouse, databases, schemas.

In [ ]:
create warehouse DBT_WH;
create database raw;
create database DBT_analytics;
create schema raw.cfa_data;
create schema raw.grobid_data;

Creating the required schemas and copying the tables formt he AWS bucket to the Snowflake

In [ ]:
copy into raw.cfa_data.CFA_Content (ID, Topic_Name, YEAR_, Level , INTRODUCTION_SUMMARY , LEARNING_OUTCOMES , SUMMARY_PAGE_LINK , PDF_FILE_LINK)

from 's3://assign2amos/Grobid_CSV/GrobidContent_dataSet.csv_0_0_0.csv.gz'
file_format = (
    type = 'CSV'
    field_delimiter = ','
    skip_header = 1
    );

In [ ]:
copy into raw.cfa_data.CFA_Content (ID, Topic_Name, YEAR_, Level , INTRODUCTION_SUMMARY , LEARNING_OUTCOMES , SUMMARY_PAGE_LINK , PDF_FILE_LINK)

from 's3://assign2amos/Grobid_CSV/GrobidMetaData_dataSet.csv_0_0_0.csv.gz'
file_format = (
    type = 'CSV'
    field_delimiter = ','
    skip_header = 1
    );

In [ ]:
copy into raw.cfa_data.CFA_Content (ID, Topic_Name, YEAR_, Level , INTRODUCTION_SUMMARY , LEARNING_OUTCOMES , SUMMARY_PAGE_LINK , PDF_FILE_LINK)

from 's3://assign2amos/CFA_CSV/CFAcontent_dataSet.csv_0_0_0.csv.gz'

file_format = (
    type = 'CSV'
    field_delimiter = ','
    skip_header = 1
    );

In [ ]:
Verified the changes using the select query

###Connecting DBT cloud to SNF - Manually



Create a new project in dbt Cloud. From Account settings (using the gear menu in the top right corner), click + New Project.

Enter a project name and click Continue.

For the warehouse, click Snowflake then Next to set up your connection.


Enter your **Settings** for Snowflake.

Enter your **Development Credentials** for Snowflake.

Verify the Connection - TEST CONNECTION

###Setup a DBT cloud management repository

Under "Setup a repository", select Managed.
Type a name for your repo such as BigData-Assignment3
Click Create.



##Initialize and start of DBT project

###Building the first Model:

In [ ]:
WITH cfa_content_dataset AS (
    SELECT
        Level,
        Topic_Name,
        Year_,
        LENGTH(INTRODUCTION_SUMMARY) AS Summary_Length,
        LENGTH(LEARNING_OUTCOMES) AS Learning_Outcomes_Length,
        NULL AS file_path,
        NULL AS Number_of_Articles
    FROM {{ ref('stg_CFA') }}
),
grobidmetadata_new AS (
    SELECT
        NULL AS Level,
        NULL AS Topic,
        NULL AS Year,
        NULL AS Summary_Length,
        NULL AS Learning_Outcomes_Length,
        file_path,
        number_of_articles
    FROM {{ ref('stg_grobidMetaData') }}
),
combined_data AS (
    SELECT * FROM cfa_content_dataset
    UNION ALL
    SELECT * FROM grobidmetadata_new
)

SELECT
    Level,
    Topic_Name,
    Year_,
    MIN(Summary_Length) AS Min_Length_Summary,
    MAX(Summary_Length) AS Max_Length_Summary,
    MIN(Learning_Outcomes_Length) AS Min_Length_Learning_Outcomes,
    MAX(Learning_Outcomes_Length) AS Max_Length_Learning_Outcomes,
    file_path,
    number_of_articles
FROM combined_data
GROUP BY Level, Topic_Name, Year_, file_path, number_of_articles

In [ ]:
#run in the terminal
dbt run

In [ ]:
#materailzation of the model created above
--------------
{{
  config(
    materialized='view'
  )
}}
------------------
WITH cfa_content_dataset AS (
    SELECT
        Level,
        Topic_Name,
        Year_,
        LENGTH(INTRODUCTION_SUMMARY) AS Summary_Length,
        LENGTH(LEARNING_OUTCOMES) AS Learning_Outcomes_Length,
        NULL AS file_path,
        NULL AS Number_of_Articles
    FROM {{ ref('stg_CFA') }}
),
grobidmetadata_new AS (
    SELECT
        NULL AS Level,
        NULL AS Topic,
        NULL AS Year,
        NULL AS Summary_Length,
        NULL AS Learning_Outcomes_Length,
        file_path,
        number_of_articles
    FROM {{ ref('stg_grobidMetaData') }}
),
combined_data AS (
    SELECT * FROM cfa_content_dataset
    UNION ALL
    SELECT * FROM grobidmetadata_new
)

SELECT
    Level,
    Topic_Name,
    Year_,
    MIN(Summary_Length) AS Min_Length_Summary,
    MAX(Summary_Length) AS Max_Length_Summary,
    MIN(Learning_Outcomes_Length) AS Min_Length_Learning_Outcomes,
    MAX(Learning_Outcomes_Length) AS Max_Length_Learning_Outcomes,
    file_path,
    number_of_articles
FROM combined_data
GROUP BY Level, Topic_Name, Year_, file_path, number_of_articles

###Building models on top of sources

In [ ]:
#stg_CFA


select
    ID as ID,
    Topic_Name as Topic_Name,
    Year as Year_,
    Level as Level,
    INTRODUCTION_SUMMARY as INTRODUCTION_SUMMARY,
    LEARNING_OUTCOMES as LEARNING_OUTCOMES,
    SUMMARY_PAGE_LINK as SUMMARY_PAGE_LINK,
    PDF_FILE_LINK as PDF_FILE_LINK

from {{ source('cfa_data', 'CFA_CONTENT_DATASET') }}

In [ ]:
#stg_grobidMetadata


-- stg_grobidMetaData.sql

SELECT
    file_path AS file_path,
    FILE_SIZE AS file_size,
    NUMBER_OF_ARTICLES AS number_of_articles,
    CREATION_TIME AS creation_time,
    MODIFICATION_TIME AS modification_time,
    ENCODING_LANGUAGE AS encoding_language
FROM {{ source('grobid_data','GROBIDMETADATA_NEW') }}


In [ ]:
#contents.sql



{{
  config(
    materialized='view'
  )
}}

WITH cfa_content_dataset AS (
    SELECT
        Level,
        Topic_Name,
        Year_,
        LENGTH(INTRODUCTION_SUMMARY) AS Summary_Length,
        LENGTH(LEARNING_OUTCOMES) AS Learning_Outcomes_Length,
        NULL AS file_path,
        NULL AS Number_of_Articles
    FROM {{ ref('stg_CFA') }}
),
grobidmetadata_new AS (
    SELECT
        NULL AS Level,
        NULL AS Topic,
        NULL AS Year,
        NULL AS Summary_Length,
        NULL AS Learning_Outcomes_Length,
        file_path,
        number_of_articles
    FROM {{ ref('stg_grobidMetaData') }}
),
combined_data AS (
    SELECT * FROM cfa_content_dataset
    UNION ALL
    SELECT * FROM grobidmetadata_new
)

SELECT
    Level,
    Topic_Name,
    Year_,
    MIN(Summary_Length) AS Min_Length_Summary,
    MAX(Summary_Length) AS Max_Length_Summary,
    MIN(Learning_Outcomes_Length) AS Min_Length_Learning_Outcomes,
    MAX(Learning_Outcomes_Length) AS Max_Length_Learning_Outcomes,
    file_path,
    number_of_articles
FROM combined_data
GROUP BY Level, Topic_Name, Year_, file_path, number_of_articles


creating source.yml files for building the models on top of the sources

Sources make it possible to name and describe the data loaded into your warehouse by your extract and load tools.

select from source tables in your models using the {{ source() }} function, helping define the lineage of your data

In [ ]:
#source.yml


version: 2

sources:
  - name: cfa_data
    description: This is a replica of the Postgres database used by our app
    database: raw
    schema: cfa_data
    tables:
      - name: CFA_CONTENT_DATASET
        description: contains file contents extracted from CFA using beautifulsoup [webscraping]

  - name: grobid_data
    description: Grobid data schema
    database: raw
    schema: grobid_data
    tables:
      - name: GROBIDMETADATA_NEW
        description: contains metadata of grobid files
      - name: GROBIDCONTENT
        description: contains file contents extracted from Grobid



###implementing testing


In [ ]:
#schema.yml


version: 2

models:
  - name: cfa_contents
    columns:
      - name: Level
        tests:
          - not_null
        description: NULL when a level is missing
      - name: Topic
        tests:
          - not_null
        description: NULL when a Topic is missing
     Initialize your dbt project​ and start developing
Now that you have a repository configured, you can initialize your project and start development in dbt Cloud:
          -  range_check:
              start: 2000
              end: 2050
        description: the year should range bet 2000 - 2050

      - name: Min_Length_Summary
        tests:
          - not_null
        description: gives min-length_summary
      - name: Max_Length_Summary
        tests:
          - not_null
        description: gives max-length_summary
      - name: Min_Length_Learning_Outcomes
        tests:
          - not_null
        description: gives min-length_LO
      - name: Max_Length_Learning_Outcomes
        tests:
          - not_null
        description: gives max-length_LO

  - name: stg_cfaContent
    columns:
      - name: ID
        tests:
          - unique
          - not_null

      - name: Topic_Name
        tests:
          - not_null
      - name: YEAR_
        tests:
          - not_null
      - name: Level
        tests:
          - not_null
      - name: INTRODUCTION_SUMMARY
        tests:
          - not_null
      - name: LEARNING_OUTCOMES
        tests:
          - not_null
      - name: SUMMARY_PAGE_LINK
        tests:
          - not_null
      - name: PDF_FILE_LINK
        tests:
          - not_null

  - name: stg_grobidMetaData
    columns:
      - name: file_path
        tests:
          - unique
          - not_null
      - name: file_size
        tests:
          - not_null
      - name: number_of_articles
        tests:
          - not_null
      - name: creation_time
        tests:
          - not_null
      - name: modification_time
        tests:
          - not_null
      - name: encoding_language
        tests:
          - not_null


In [ ]:
#run to check the test case outcomes
dbt test

After successful testing, commit the changes and merge to main branch


###Deploy DBT

environments are created for deployment

* test
* Prod